In [1]:
 import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load and preprocess the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

x_train = x_train.astype('float32') / 255.0
x_val = x_val.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = to_categorical(y_train, 10)
y_val = to_categorical(y_val, 10)
y_test = to_categorical(y_test, 10)

# Define a function to create a CNN model
def create_cnn_model(learning_rate=0.001, filter_size=32, num_layers=3, dropout_rate=0.5, optimizer='adam'):
    model = models.Sequential()
    model.add(layers.Conv2D(filter_size, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(layers.MaxPooling2D((2, 2)))

    for _ in range(num_layers - 1):
        model.add(layers.Conv2D(filter_size * 2, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        filter_size *= 2

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(10, activation='softmax'))

    if optimizer == 'adam':
        opt = optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = optimizers.SGD(learning_rate=learning_rate)
    else:
        raise ValueError("Unsupported optimizer type")

    model.compile(optimizer=opt,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create and train the model
model = create_cnn_model(learning_rate=0.001, filter_size=32, num_layers=3, dropout_rate=0.5, optimizer='adam')
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val))

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 19ms/step - accuracy: 0.2553 - loss: 1.9911 - val_accuracy: 0.4771 - val_loss: 1.4392
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 13s 21ms/step - accuracy: 0.4567 - loss: 1.5022 - val_accuracy: 0.5527 - val_loss: 1.2368
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 14s 23ms/step - accuracy: 0.5231 - loss: 1.3332 - val_accuracy: 0.5813 - val_loss: 1.1436
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.5622 - loss: 1.2266 - val_accuracy: 0.6130 - val_loss: 1.0814
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 15s 25ms/step - accuracy: 0.6033 - loss: 1.1309 - val_accuracy: 0.6350 - val_loss: 1.0466
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 19s 30ms/step - accuracy: 0.6288 - loss: 1.0690 - val_accuracy: 0.6556 - val_loss: 0.9752
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 20s 31ms/step - accuracy: 0.6469 - loss: 1.0010 - val_accuracy: 0.6787 - val_loss: 0.9135
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 23s 36ms/step - accuracy: 0.6726 - loss: 0.9443 - 

In [2]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=0)
print(f'Test accuracy: {test_acc:.4f}')

# Make predictions on the test set
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Print classification report
print("Classification Report:")
print(classification_report(y_true, y_pred_classes))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred_classes))


Test accuracy: 0.6990
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.78      0.76      1000
           1       0.80      0.85      0.82      1000
           2       0.62      0.56      0.59      1000
           3       0.54      0.42      0.47      1000
           4       0.65      0.64      0.64      1000
           5       0.53      0.69      0.60      1000
           6       0.72      0.80      0.76      1000
           7       0.73      0.77      0.75      1000
           8       0.86      0.79      0.82      1000
           9       0.84      0.70      0.76      1000

    accuracy                           0.70     10000
   macro avg       0.70      0.70      0.70     10000
weighted avg       0.70      0.70      0.70     10000

Confusion Matrix:
[[781  29  61  11  18   3  12  10  48  27]
 [ 23 848   7   7   6   8  17   7  19  58]
 [ 73   7 555  48  98 107  69  31   8   4]
 [ 15   